In [2]:
import pandas as pd
import requests
import re
import json
from bs4 import BeautifulSoup

In [26]:
def get_soup(offset=1, position=0, stat_type='seasonStats', stat_season=2022, stat_week=1):
    url = 'https://fantasy.nfl.com/league/3967921/players'
    params = {
        'offset': offset,
        'playerStatus': 'all',
        'position': position,
        'statCategory': 'stats',
        'statType': stat_type,
        'statSeason': stat_season,
        'statWeek': stat_week,
        'sort': 'pts',
    }
    header = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
    }
    r = requests.get(url, headers=header, params=params)
    print(r)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

soup = get_soup(position=12)
print(soup.prettify())   

<Response [200]>
<!DOCTYPE html>
<!--[if lt IE 7]>  <html class="ie ie6 lte9 lte8 lte7 skinId-1"  lang="en" id="template" > <![endif]-->
<!--[if IE 7]>     <html class="ie ie7 lte9 lte8 lte7 gt6 skinId-1" lang="en" id="template"> <![endif]-->
<!--[if IE 8]>     <html class="ie ie8 lte9 lte8 gt6 gt7 skinId-1" lang="en" id="template"> <![endif]-->
<!--[if IE 9]>     <html class="ie ie9 lte9 gt6 gt7 gt8 skinId-1" lang="en" id="template"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!-->
<html class="not-ie skinId-1" id="template" itemscope="" itemtype="http://schema.org/Organization" lang="en">
 <!--<![endif]-->
 <head>
  <!-- app dynamics random number = 2 -->
  <script charset="utf-8" src="/static/js/1694025258/bW9kZXJuaXpyLmpz.js">
  </script>
  <script src="/min/index1?a=230443d8-649d-11e6-8b77-86f30ca893d3&amp;g=ff-base">
  </script>
  <link id="nfl-assets"/>
  <script type="text/javascript">
   var googletag = googletag || {};
    googletag.cmd = googletag.cmd || [];
    googletag.cmd.pus

In [32]:
def scrape_stat_meta(soup):
    # search for <th aria-label="Completions" class="stat stat_3 numeric" scope="col">
    stat_items = soup.find_all('th', {'class': re.compile('stat stat_')})
    stat_meta_list = list()
    for e in stat_items:
        stat_id = e.attrs['class'][1]
        ce = e.find('a')
        if ce:
            abbr = ce.text
        else:
            abbr = None
        d = {
            'stat_id': stat_id,
            'stat_type': e.attrs['class'][2],
            'label': e.attrs.get('aria-label', stat_id),
            'abbr': abbr
        }
        stat_meta_list.append(d)
    return stat_meta_list
   
df_list = list()
base_categories = {
    'offense': 0,
    'defense': 12,
    'kicker': 7
}

for bc, pos in base_categories.items():
    soup = get_soup(position=pos)
    sml = scrape_stat_meta(soup)
    df = pd.DataFrame(data=sml)
    df['base_category'] = bc
    df_list.append(df)
df_stats_meta = pd.concat(df_list, ignore_index=False)
df_stats_meta.reset_index(inplace=True, drop=True)
#df_stats_meta

<Response [200]>
[   Unnamed: 0_level_0             Unnamed: 1_level_0 Unnamed: 2_level_0   
               Action                         Player                Opp   
0                 NaN        Patrick Mahomes QB - KC               @ARI  \
1                 NaN            Josh Allen QB - BUF                @LA   
2                 NaN  Joe Burrow QB - CIN View News                PIT   
3                 NaN        Justin Herbert QB - LAC                 LV   
4                 NaN           Jalen Hurts QB - PHI               @DET   
5                 NaN          Kirk Cousins QB - MIN                 GB   
6                 NaN            Geno Smith QB - SEA                DEN   
7                 NaN            Jared Goff QB - DET                PHI   
8                 NaN       Trevor Lawrence QB - JAX               @WAS   
9                 NaN          Daniel Jones QB - NYG               @TEN   
10                NaN        Tua Tagovailoa QB - MIA                 NE   
11     

In [43]:
def scrape_stats_table(soup):
    # search for <div class="tableWrap">:
    div_table = soup.find('div', {'class': 'tableWrap'})
    if div_table:
        scrape_res = pd.read_html(str(div_table))
        df = scrape_res[0]
        df.columns = ['_'.join(col) for col in df.columns.values]
        return df[0]
    else:
        return None
    
position_mapping = {
    'QB': 1,
    'RB': 2,
    'WR': 3,
    'TE': 4,
    'DL': 11,
    'LB': 12,
    'DB': 13,
    'K': 7
}
df_list = []

for pos, pos_nb in position_mapping.items():
    offset = 1
    while offset > 0:
        soup = get_soup(position=pos_nb, offset=offset)
        df = scrape_stats_table(soup)
        if df is not None:
            df_list.append(df)
            offset += 25
        else:
            offset = 0
data = pd.concat(df_list, ignore_index=True)
data.reset_index(inplace=True, drop=True)
data

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

,Unnamed: 0_level_0_Action,Unnamed: 1_level_0_Player,Unnamed: 2_level_0_Opp,Unnamed: 3_level_0_Manager,Passing_Comp,Passing_Yds,Passing_TD,Passing_Int,Rushing_Yds,Rushing_TD,...,FG Made_0-19,FG Made_20-29,FG Made_30-39,FG Made_40-49,FG Made_50+,FG Miss_0-19,FG Miss_20-29,FG Miss_30-39,FG Miss_40-49,FG Miss_50+
0,NaN,Patrick Mahomes QB - KC,@ARI,St Quintin Heidschnucken,435,5250,41,12,358,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Josh Allen QB - BUF,@LA,FlyRamsFly,359,4283,35,14,762,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Joe Burrow QB - CIN View News,PIT,Magic Eulers,414,4475,35,12,257,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Justin Herbert QB - LAC,LV,Bad Wildcats Hunschat,477,4739,25,10,147,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Jalen Hurts QB - PHI,@DET,dirty birds,306,3701,22,6,760,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2168,NaN,Giorgio Tavecchio K,Bye,FA,NaN,NaN,NaN,NaN,NaN,NaN,...,-,-,-,-,-,-,-,-,-,-
2169,NaN,Matthew Trickett K,Bye,FA,NaN,NaN,NaN,NaN,NaN,NaN,...,-,-,-,-,-,-,-,-,-,-
2170,NaN,Jake Verity K,@DAL,FA,NaN,NaN,NaN,NaN,NaN,NaN,...,-,-,-,-,-,-,-,-,-,-
2171,NaN,Parker White K,Bye,FA,NaN,NaN,NaN,NaN,NaN,NaN,...,-,-,-,-,-,-,-,-,-,-


In [44]:
data.to_excel('player_sztats.xlsx', index = False)

In [36]:
url = 'https://fantasy.nfl.com/league/3967921/players?offset=26&playerStatus=all&position=O&sort=pts&sortOrder=desc&statCategory=stats&statSeason=2022&statType=seasonStats&statWeek=1'
#url = 'https://fantasy.nfl.com/league/3967921/players?offset=52&playerStatus=all'
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)
print(r)
soup = BeautifulSoup(r.text, 'html.parser')
print(soup.prettify())

<Response [200]>
<!DOCTYPE html>
<!--[if lt IE 7]>  <html class="ie ie6 lte9 lte8 lte7 skinId-1"  lang="en" id="template" > <![endif]-->
<!--[if IE 7]>     <html class="ie ie7 lte9 lte8 lte7 gt6 skinId-1" lang="en" id="template"> <![endif]-->
<!--[if IE 8]>     <html class="ie ie8 lte9 lte8 gt6 gt7 skinId-1" lang="en" id="template"> <![endif]-->
<!--[if IE 9]>     <html class="ie ie9 lte9 gt6 gt7 gt8 skinId-1" lang="en" id="template"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!-->
<html class="not-ie skinId-1" id="template" itemscope="" itemtype="http://schema.org/Organization" lang="en">
 <!--<![endif]-->
 <head>
  <!-- app dynamics random number = 3 -->
  <script charset="utf-8" src="/static/js/1693954783/bW9kZXJuaXpyLmpz.js">
  </script>
  <script src="/min/index1?a=230443d8-649d-11e6-8b77-86f30ca893d3&amp;g=ff-base">
  </script>
  <link id="nfl-assets"/>
  <script type="text/javascript">
   var googletag = googletag || {};
    googletag.cmd = googletag.cmd || [];
    googletag.cmd.pus

In [37]:
stat_lookup = dict()
player_list = soup.find_all('tr', {'class': re.compile('player-*')})
for p in player_list:
    player_id = p.attrs['class'][0]
    #print(p)
    pne = p.find('a', {'class': re.compile('playerCard playerName playerNameFull ')})
    player_name = pne.text
    em = p.find('em')
    position = em.text.split("-")[0].strip()
    team = em.text.split("-")[1].strip()
    owner = p.find('td', {'class':"playerOwner"}).text
    opponent = p.find('td', {'class': "playerOpponent"}).text
        
    d = {'player_id': player_id,
         'player_name': player_name,
         'position': position,
         'team': team,
         'owner': owner,
         'opponent': opponent}
    
    stat_list = p.find_all('td', {'class': re.compile('stat stat_')})
    player_stats = list()
    for st in stat_list:
        stat_id = st.attrs['class'][1]
        stat_type = st.attrs['class'][2]
        stat_name = stat_lookup.get(stat_id, None)
        stat_value = st.find('span').text           
        s = {'stat_id': stat_id,
             'stat_type': stat_type,
             'stat_name': stat_name,
             'stat_value': stat_value}
        player_stats.append(s)
    d['stats'] = player_stats
    print(d)

{'player_id': 'player-2543801', 'player_name': 'Jimmy Garoppolo', 'position': 'QB', 'team': 'LV', 'owner': 'FA', 'opponent': '@LAC', 'stats': [{'stat_id': 'stat_3', 'stat_type': 'numeric', 'stat_name': None, 'stat_value': '207'}, {'stat_id': 'stat_5', 'stat_type': 'numeric', 'stat_name': None, 'stat_value': '2437'}, {'stat_id': 'stat_6', 'stat_type': 'numeric', 'stat_name': None, 'stat_value': '16'}, {'stat_id': 'stat_7', 'stat_type': 'numeric', 'stat_name': None, 'stat_value': '4'}, {'stat_id': 'stat_14', 'stat_type': 'numeric', 'stat_name': None, 'stat_value': '33'}, {'stat_id': 'stat_15', 'stat_type': 'numeric', 'stat_name': None, 'stat_value': '2'}, {'stat_id': 'stat_20', 'stat_type': 'numeric', 'stat_name': None, 'stat_value': '-'}, {'stat_id': 'stat_21', 'stat_type': 'numeric', 'stat_name': None, 'stat_value': '-'}, {'stat_id': 'stat_22', 'stat_type': 'numeric', 'stat_name': None, 'stat_value': '-'}, {'stat_id': 'stat_27', 'stat_type': 'numeric', 'stat_name': None, 'stat_value': 